<a href="https://colab.research.google.com/github/0xVolt/learn-langchain/blob/main/hf_models_locally_with_langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HuggingFace Models Locally with LangChain

## Ways to Create HuggingFace Models

There are two wrappers that HuggingFace (HF) provides to use LLMs, one for a local run and host and another for hosting your models and projects on HuggingFaceHub (HFH). You'll see most projects using the latter approach. However, there are pros and cons for both.

## Why not `HuggingFaceHub`?

Plain and simple, the HFH doesn't support all models on HF. And, for the models that are supported, the wrappers only exist for the following tasks: `text2text-generation` and `text-generation`. So, I've decided to against using HFH for this notebook and these models.

### More on the Tasks

`text2text-generation` represents tasks that are accomplished by encoder-decoder models like BERT, T5 and BART.

`text-generation` represents tasks that are done by models that are solely decoders like GPT-2, etc.

It's important to understand what kind of model you're working with since the definition and parameters that you'd use to setup the LangChain will differ. Examples of this are to follow.

## Why Locally?

You have the ability to fine-tune any of these models, get complete access to your GPU (if you have one) and avoid running into issues with models that don't run on the HFH.

## About this Notebook

This notebook will compare different models by setting them up locally and then linking them to LangChain. The idea is to use this notebook for future reference if I need to make a project that involves fine-tuning any of these models to create a LangChain backed application.

---

In [3]:
!pip -q install langchain transformers sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 97.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 12.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 103.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 88.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 68.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.6 MB/s eta 0:00:00


# Model #1 - `T5-Flan`

## Create the LLM

This model us an encoder-decoder so we'll use the `text2text-generation` task to initialise it as I'd mentioned earlier. Notice how we import `AutoModelForSeq2SeqLM`. Pro tip: `Seq2Seq` is just another term for encoder-decoder.

In [17]:
import torch
from langchain import PromptTemplate, LLMChain
from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer, pipeline, AutoModelForSeq2SeqLM, AutoModelForCausalLM

Initialise the model and the auto-tokenizer.

In [5]:
modelID = 'google/flan-t5-large'
tokenizer = AutoTokenizer.from_pretrained(modelID)
model = AutoModelForSeq2SeqLM.from_pretrained(modelID)

Create the local LLMs pipeline with the model and tokenizer we defined.

In [6]:
pipe = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=100
)

Finally, instantiate the LLM.

In [7]:
localLLM = HuggingFacePipeline(pipeline=pipe)

## Feed the LLM Prompts

Create a question and answer template using the `PromptTemplate` module.

In [23]:
template = """Question: {question}

Answer: Let's think step by step."""

prompt = PromptTemplate(template=template, input_variables=["question"])

Next, create the LLM chain which takes a prompt template and the LLM to use.

In [24]:
localLLMChain = LLMChain(
    prompt=prompt,
    llm=localLLM
)

Finally, put it all together by running the LLM chain.

In [25]:
# question = input("Question: ")
question = "What is India's primary export?"

In [26]:
print(localLLMChain.run(question))

India's primary export is textiles. Textiles are made of cotton. Cotton is a staple fabric in India. Cotton is a staple fabric in the United States. So the answer is cotton.


---